## Modelo Avanzado

https://www.kaggle.com/code/ryanholbrook/forecasting-with-machine-learning

### Seteando entorno

In [1]:
# Set-up librerias
from helper import *
librerias = ['numpy', 'pandas', 'matplotlib', 'pathlib', 'seaborn', 'sklearn', 'statsmodels', 'kaggle']
load_install_package(librerias)

numpy está instalada y lista para usar.
pandas está instalada y lista para usar.
matplotlib está instalada y lista para usar.
pathlib está instalada y lista para usar.
seaborn está instalada y lista para usar.
sklearn está instalada y lista para usar.
statsmodels está instalada y lista para usar.
kaggle está instalada y lista para usar.


In [2]:
# Cargamos las librerias
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import kaggle
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.metrics import mean_squared_log_error

### Cargando Data

In [7]:
store_sales = pd.read_csv(
    '../kgl_Store_Sales/Data/train.csv',
    
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['date']).sort_index()

family_sales = (
    store_sales
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

test = pd.read_csv('../kgl_Store_Sales/Data/test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
test['date'] = test.date.dt.to_period('D')
test = test.set_index(['date']).sort_index()

/var/folders/mv/dc1zj8q116s9fq6jrlqb43040000gn/T/ipykernel_8501/1902033189.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .mean()


### Crossvalidation Splits

In [8]:
# Genero los splits
ventana_train = 100
ventana_test = 15
btss = BlockingTimeSeriesSplit(train_size=ventana_train, test_size=ventana_test, step=ventana_train + ventana_test)
df_cv = sliding_period(df = store_sales, btss = btss, period = "day")
df_cv[0]

,id,store_nbr,family,sales,onpromotion,split
date,,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.00000,0,train
2013-01-01,1,1,BABY CARE,0.00000,0,train
2013-01-01,2,1,BEAUTY,0.00000,0,train
2013-01-01,3,1,BEVERAGES,0.00000,0,train
2013-01-01,4,1,BOOKS,0.00000,0,train
...,...,...,...,...,...,...
2013-04-25,204925,9,POULTRY,395.60199,0,test
2013-04-25,204926,9,PREPARED FOODS,65.00000,0,test
2013-04-25,204927,9,PRODUCE,0.00000,0,test
